In [1]:
import os 


## Model

In [2]:
from langchain.chat_models import ChatOpenAI

llm_model = ''

chat = ChatOpenAI(temperature = 0, 
                  model = llm_model)



ModuleNotFoundError: No module named 'langchain_community'

## Prompt Template

In [3]:
from langchain.prompts import ChatPromptTemplate
#text = ''
#style = 'O mesmo estilo de fala que o batman teria, o tom de batman sério'
template_string = """
    Trauza o texto delimitado por "",
    para o estilo {style}
    "{text}", também complete a técnica secreta de acordo com o personagem 
     citado no style """

text = ''' 
    Voce nao poderia me derrotar, pois por ventura, sou deveras
    mais poderoso que ti hihhihi, agora contemple minha tecnica secreta: 
'''

style = ''' Vegetta muito furioso'''

prompt_template = ChatPromptTemplate.from_template(template_string)
formatPrompt = prompt_template.format_messages(
    style = style, 
    text = text 
)

print(formatPrompt[0].content)




    Trauza o texto delimitado por "",
    para o estilo  Vegetta muito furioso
    " 
    Voce nao poderia me derrotar, pois por ventura, sou deveras
    mais poderoso que ti hihhihi, agora contemple minha tecnica secreta: 
", também complete a técnica secreta de acordo com o personagem 
     citado no style 


In [4]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], input_types={}, partial_variables={}, template='\n    Trauza o texto delimitado por "",\n    para o estilo {style}\n    "{text}", também complete a técnica secreta de acordo com o personagem \n     citado no style ')

### Output Parsers

A way that let you define how python can be deal with the ouput of model. 

Example: I want that my output have a json format... returns needs to be a python dictionary 

* Gonna be create a parser to a json format, with the keys:

    * gift:
    * deleivery_days
    * prince_value

In [ ]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser 

In [9]:
# create one schema for each key in the output
# O schema description é puramente documental, ou possui 
# alguma finalidade de orientações ao modelo?

gift_schema = ResponseSchema(name = 'gift',
                             description= ("Was the item purchased as a gift for someone else?"
                                            "response True for yse, "
                                            "False if not or unknown")
                                            )


delivery_days_schema = ResponseSchema(
    name = 'delivery_days',
    description= ("How many days didi it take for the product to arrive?"
                  "If this information is not found, output -1")
    )


price_value_schema = ResponseSchema(
    name = 'price_value',
    description = 'Extract any sentences about the value or price, and out them as a comma separated Python list'
)

responseSchemas = [gift_schema, delivery_days_schema, price_value_schema]


In [ ]:
# Cria uma "tupla" com "n" elementos, um para cada schema/key definido.

output_parser = StructuredOutputParser.from_response_schemas(responseSchemas)
print(output_parser)

response_schemas=[ResponseSchema(name='gift', description='Was the item purchased as a gift for someone else?response True for yse, False if not or unknown', type='string'), ResponseSchema(name='delivery_days', description='How many days didi it take for the product to arrive?If this information is not found, output -1', type='string'), ResponseSchema(name='price_value', description='Extract any sentences about the value or price, and out them as a comma separated Python list', type='string')]


In [12]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased as a gift for someone else?response True for yse, False if not or unknown
	"delivery_days": string  // How many days didi it take for the product to arrive?If this information is not found, output -1
	"price_value": string  // Extract any sentences about the value or price, and out them as a comma separated Python list
}
```


In [16]:
customer_review = 'blablabla'
template_string = '''
    Im a template string with the following paramters:
    
    text: {text}

    {format_instructions}

 '''

prompt = ChatPromptTemplate.from_template(template = template_string)
messages = prompt.format_messages(
    text = customer_review,
    format_instructions=format_instructions
)

print(messages[0].content)


    Im a template string with the following paramters:

    text: blablabla

    The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased as a gift for someone else?response True for yse, False if not or unknown
	"delivery_days": string  // How many days didi it take for the product to arrive?If this information is not found, output -1
	"price_value": string  // Extract any sentences about the value or price, and out them as a comma separated Python list
}
```

 


In [ ]:
# invocar o modelo com o parser...
response = chat(messages)

In [ ]:
# apos o modelo processar o conteudo, utilizar o parser para a atividade de parsing do output
output_parsed = output_parser.parse(response.content)


## Memory for chat bots

In [ ]:
from langchain_google_genai     import ChatGoogleGenerativeAI 
from langchain.chains           import ConversationChain
from langchain.memory           import ConversationBufferMemory 
import os 
from dotenv import load_dotenv, find_dotenv 
load_dotenv()
api_key = os.getenv('GOOGLE_API_KEY')
llm = ChatGoogleGenerativeAI(model='gemini-2.0-flash-lite-001')


AIMessage(content='Tudo bem por aqui! 😊 E com você? Como posso te ajudar hoje?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash-lite-001', 'safety_ratings': []}, id='run--10e169c4-c111-41fd-b698-81a402b1dbc2-0', usage_metadata={'input_tokens': 3, 'output_tokens': 17, 'total_tokens': 20, 'input_token_details': {'cache_read': 0}})

In [12]:
memory = ConversationBufferMemory()
chatBot = ConversationChain(
    llm = llm, 
    memory = memory, 
    verbose = True #If true, tracking the llm execution pipeline 
)


C:\Users\ricar\AppData\Local\Temp\ipykernel_3184\3770664563.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()
C:\Users\ricar\AppData\Local\Temp\ipykernel_3184\3770664563.py:2: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  chatBot = ConversationChain(


In [14]:
chatBot.predict(input="hi my name is Ricardo")




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: hi my name is Ricardo
AI:

> Finished chain.


"Hello Ricardo! It's a pleasure to meet you. My name is Bard, and I'm a large language model, trained by Google. I'm designed to be informative and comprehensive. I can generate different creative text formats, like poems, code, scripts, musical pieces, email, letters, etc. I try my best to follow your instructions and complete your requests thoughtfully. I'm ready when you are! What can I help you with today, Ricardo? Do you have any questions, or are you looking for something specific? Perhaps you'd like me to tell you a joke, write a short story, or maybe even translate something? Just let me know!"

In [17]:
print( chatBot.predict(input = 'How many r´s are in strawberry?'))



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: hi my name is Ricardo
AI: Hello Ricardo! It's a pleasure to meet you. My name is Bard, and I'm a large language model, trained by Google. I'm designed to be informative and comprehensive. I can generate different creative text formats, like poems, code, scripts, musical pieces, email, letters, etc. I try my best to follow your instructions and complete your requests thoughtfully. I'm ready when you are! What can I help you with today, Ricardo? Do you have any questions, or are you looking for something specific? Perhaps you'd like me to tell you a joke, write a short story, or maybe even translate something? Just let me know!
Human: How many r´s are in strawberr

In [19]:
chatBot.predict(input = 'Whats is my name?')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: hi my name is Ricardo
AI: Hello Ricardo! It's a pleasure to meet you. My name is Bard, and I'm a large language model, trained by Google. I'm designed to be informative and comprehensive. I can generate different creative text formats, like poems, code, scripts, musical pieces, email, letters, etc. I try my best to follow your instructions and complete your requests thoughtfully. I'm ready when you are! What can I help you with today, Ricardo? Do you have any questions, or are you looking for something specific? Perhaps you'd like me to tell you a joke, write a short story, or maybe even translate something? Just let me know!
Human: How many r´s are in strawberr

"Your name is Ricardo! I remember you told me that at the beginning of our conversation. Is there anything else I can help you with, Ricardo? Perhaps you'd like me to write a short poem about strawberries, since we were just talking about them? Or maybe you'd like to know some interesting facts about other fruits? I'm ready when you are!"

### Window Memory

In [20]:
from langchain.memory import ConversationBufferWindowMemory


In [21]:
# Definindo o quao "larga" sera a janela de contexto (em numero de past messages)
memory = ConversationBufferWindowMemory(k=1)
chatBot = ConversationChain(
    llm = llm , 
    memory = memory, 
    verbose = True
)

C:\Users\ricar\AppData\Local\Temp\ipykernel_3184\3143180504.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=1)


In [22]:
chatBot.predict(input = 'My name is Ricardo')




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: My name is Ricardo
AI:

> Finished chain.


"Hello Ricardo! It's a pleasure to meet you. I'm an AI, a large language model trained by Google. I don't have a name, but you can call me anything you like! It's nice to be chatting with you today. Is there anything I can help you with? Perhaps you have a question about something, or maybe you just want to chat? I'm ready for whatever you have in mind! I can access and process information from the real world through Google Search and keep my response consistent with search results. So, just let me know what's on your mind!"

In [23]:
chatBot.predict(input = 'How many r´s are in strawberry')




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: My name is Ricardo
AI: Hello Ricardo! It's a pleasure to meet you. I'm an AI, a large language model trained by Google. I don't have a name, but you can call me anything you like! It's nice to be chatting with you today. Is there anything I can help you with? Perhaps you have a question about something, or maybe you just want to chat? I'm ready for whatever you have in mind! I can access and process information from the real world through Google Search and keep my response consistent with search results. So, just let me know what's on your mind!
Human: How many r´s are in strawberry
AI:

> Finished chain.


'That\'s a fun question, Ricardo! Let\'s find out together. Based on my knowledge, which includes information from Google Search, the word "strawberry" has **three** "r"s. '

In [24]:
chatBot.predict(input = 'Whats my name?')



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: How many r´s are in strawberry
AI: That's a fun question, Ricardo! Let's find out together. Based on my knowledge, which includes information from Google Search, the word "strawberry" has **three** "r"s. 
Human: Whats my name?
AI:

> Finished chain.


'I do not have access to your name, Ricardo. My knowledge about you is limited to what you share in our conversation.'